<a href="https://colab.research.google.com/github/acanetti/Profiler/blob/master/SpacyNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np 
import os    
from IPython.display import clear_output
PATH ='drive/My Drive/data' 
import wordcloud
import matplotlib.pyplot as plt 
from ipywidgets import interact  
import time 
%matplotlib inline 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installation et import de SPacy pour reconnaissance d'entité nommé 

In [0]:

!pip install -U spaCy
!python -m spacy download fr
!python -m spacy download fr_core_news_md  
import spacy
from spacy import displacy
nlp = spacy.load('fr') 
clear_output()


# Import données Démocratie 

In [4]:
df=pd.read_csv(PATH + '/democratie_clean.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df

,Unnamed: 0,title,authorZipCode,Q_0,Q_1,Q_2,Q_3,Q_4,Q_5,Q_6,Q_7,Q_8,Q_9,Q_10,Q_11,Q_12,Q_13,Q_14,Q_15,Q_16,Q_17,Q_18,Q_19,Q_20,Q_21,Q_22,Q_23,Q_24,Q_25,Q_26,Q_27,Q_28,Q_29,Q_30,Q_31,Q_32,Q_33,Q_34,Q_35,Q_36
0,0,Les augmentations de rémunérations,79190,Le citoyen,Non,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,Afin d’éviter de creuser les inégalités ne plu...
1,1,rénover l'enquête publique pour en faire un vr...,1800,Un instrument de démocratie locale à modernise...,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//
2,2,Limitations de vitesse et sécurité routière,44300,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,Nous proposons le retour à la limitation de vi...
3,3,Pour un nouveau contrat citoyen,29490,Voir l'intégralité de la proposition dans la d...,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,POUR UN NOUVEAU CONTRAT CITOYEN ...
4,4,Développer les démarches de budget participati...,94300,"député, maire, moi même",Non,//,"Budget participatif, possibilité d'interpeller...",Une bonne chose,//,//,Oui,//,//,Oui,//,Des démarches de débat public (délibération) e...,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116544,116544,En finir avec la 5e république,31380,"Le milieu associatif, Les élus locaux.",Oui,Toute association ou organisation sans but luc...,Les élus devrait rendre des comptes à échéance...,Une bonne chose,Il n'est pas humainement possible de faire cor...,plus de proportionnelle,Oui,Redondances entre intercommunalités et départe...,Changer les modes de scrutin pour pouvoir réel...,Oui,Comptabiliser les votes blancs afin de pouvoir...,Le sénat devrait être supprimé et remplacé par...,Je ne sais pas,//,Proposer des référendums en ligne avec éventue...,Elle devrait pouvoir valider ou refuser les pr...,Oui,Elles devraient être remplacée par une assembl...,Appliquer les lois éxistantes semble largement...,???,Faire en sorte que les citoyens se reconnaisse...,"Entraide, bienvaillance, respect des différence.",L'éducation: Mettre en avant ces comportement ...,//,//,//,//,//,//,//,//,//,//,//,//,//
116545,116545,Démocratie,30600,//,Oui,//,//,Une bonne chose,//,//,Oui,//,//,Oui,Si le vote blanc est majoritaire d'autres list...,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,Non,//,//,//,//,//,//
116546,116546,Une démocratie plus réelle.,75012,aucun.,Non,//,//,Une mauvaise chose,Lien entre élus nationaux et population,//,Oui,Conseils de groupements intercommunaux.,//,Oui,En le comptabilisant à part.,Le référendum d'initiative citoyenne.,//,//,//,//,//,//,//,//,//,//,//,//,"Comportements agressifs, grossiers envers les ...",Verbalisation et contrainte au paiement des am...,//,//,//,Oui,Participation à des travaux d'intérêt collectif.,Situation incontrôlée. Critères : les besoins...,//,//,//,Oui. Non-représentativité des élus. Absence d...
116547,116547,Halte aux lois liberticides telle la loi anti ...,67550,en moi même!,Oui,syndicats,moins de mensonges dans les programmes. Moins ...,Une mauvaise chose,On a des députés qui n'ont jamais eu à gérer u...,Un peu de proportionnelle au niveau des législ...,Non,Plus on réduit les élus plus ils sont loin du ...,Arrêter de faire des programmes alléchants non...,Oui,Refaire l'élection si plus d'un certain pource...,//,Je ne sais pas,//,//,//,Non,Ne surtout pas supprimer le sénat!!!,On a un état suffisament laic. Et je tiens à g...,//,Mieux respecter les instances. A quoi sert de ...,//,//,//,//,//,//,//,//,Non,//,//,oui. Mais que faire contre les migrants victim...,Aider les pays sur place. Arrêter de les ruine...,Dispenser des cours de langue massifs aux migr...,Il

#  NLP 

In [0]:
test = nlp(" je suis attentive , j'aime le poulet aux oignons à Paris dans une rivière que j'aime , près de la Mairie ")

In [7]:
for ent in test.ents:
     print(ent.text, ent.label_)


Paris LOC
Mairie LOC


In [8]:

for token in test:
    print("{0}/{1} <--{2}-- {3}/{4}".format(

                                        token.text,
                                        token.tag_,

                                        token.dep_,

                                        token.head.text,

                                        token.head.tag_))


 /_SP <---- je/PRON__Number=Sing|Person=1
je/PRON__Number=Sing|Person=1 <--nsubj-- attentive/ADJ__Gender=Fem|Number=Sing
suis/AUX__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin <--cop-- attentive/ADJ__Gender=Fem|Number=Sing
attentive/ADJ__Gender=Fem|Number=Sing <--ROOT-- attentive/ADJ__Gender=Fem|Number=Sing
,/PUNCT___ <--punct-- attentive/ADJ__Gender=Fem|Number=Sing
j'/PRON__Number=Sing|Person=1 <--nsubj-- aime/VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin
aime/VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin <--parataxis-- attentive/ADJ__Gender=Fem|Number=Sing
le/DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art <--det-- poulet/NOUN__Gender=Masc|Number=Sing
poulet/NOUN__Gender=Masc|Number=Sing <--obj-- aime/VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin
aux/ADJ__Gender=Masc|Number=Plur <--amod-- poulet/NOUN__Gender=Masc|Number=Sing
oignons/NOUN__Gender=Fem|Number=Plur <--nmod-- poulet/NOUN__Gender=Masc|Number=Sing
à/ADP___ <--case-- 

In [9]:
displacy.render(test, style='dep', jupyter=True, options={'distance': 130})


# Concatenation des réponses 

In [0]:
Question = df.columns[3:]
Titre = df.columns[1]  
Zipcode=df.columns[2]
df_question = df[Question].replace('//',np.nan)

In [0]:
def cleanse_talk(): 
    talk=[]
    for i in range(df.shape[0]): 
        tmp_df = df_question.iloc[i].dropna().values
        try :  
            tmp_val=''
            for char in tmp_df: 
                charstr=str(char) 
                if charstr[-1] != '.':
                    tmp_val += charstr +'. ' 
                else : 
                    tmp_val +=  charstr + ' '
            talk.append(tmp_val)
        except :  
            talk.append('None') 
    return talk 


In [0]:
data_list=cleanse_talk()

In [13]:
for token in nlp('je suis attirante'):
    print("{0}/{1} <--{2}-- {3}/{4}".format(

                                        token.text,
                                        token.tag_,

                                        token.dep_,

                                        token.head.text,

                                        token.head.tag_))

je/PRON__Number=Sing|Person=1 <--nsubj-- attirante/ADJ__Gender=Fem|Number=Sing
suis/AUX__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin <--cop-- attirante/ADJ__Gender=Fem|Number=Sing
attirante/ADJ__Gender=Fem|Number=Sing <--ROOT-- attirante/ADJ__Gender=Fem|Number=Sing


In [14]:
nlp("je suis bon")[0].head.tag_

'ADJ__Gender=Masc|Number=Sing'

In [0]:
 def gender_finder(i,data_list):
    maxima = len(data_list) 
    try : 
        test_on_one = data_list[i] 
        doc_on_one = nlp(test_on_one) 
    except : 
        raise ValueError(f' index value must be < {maxima} ')
    gender= 'None' 
    testat = 'None'
    for token in doc_on_one : 
        if 'PRON__Number=Sing' in token.tag_ : 
            if 'ADJ__Gender=Fem' in token.head.tag_: 
                gender = 'Female'  
                testat=token.dep_
                break 
            elif 'ADJ__Gender=Masc' in token.head.tag_: 
                gender = 'Male' 
                testat=token.dep_
                break 
            else :  
                None 

    return gender

In [0]:
import multiprocessing

In [0]:
def gender_parallel(start,stop): 
    for val in range(start,stop) :    
        Dict[val] = gender_finder(val,data_list)

In [0]:
def Create_process_gender(core : int = multiprocessing.cpu_count() ) -> list : 
    manager = multiprocessing.Manager()
    global Dict 
    Dict = manager.dict() 
    
    process_list=[]
    size = len(data_list) // core 
    step = core 
    for i in range(core): 
        if i != core :
            proc=multiprocessing.Process(target=gender_parallel, args=(int(size*i),int(size*(i+1))))  
        else : 
            proc=multiprocessing.Process(target=gender_parallel, args=(int(size*i),len(data_list)-1))    

        process_list.append(proc)  
    
    return process_list

In [0]:
 def entity_finder(i,data_list):
    descriptor_entity=set([])
    maxima = len(data_list) 
    try : 
        test_on_one = data_list[i] 
        doc_on_one = nlp(test_on_one) 
    except : 
        raise ValueError(f' index value must be < {maxima} ')
    entity = doc_on_one.ents  
    for i in entity : 
        if len(i.text)> 2 : 
            if i.label_ != 'MISC':
                descriptor_entity.add((i.text,i.label_))

    return descriptor_entity

In [0]:
def parallel_entity (start,stop): 
    for  i in range(start,stop): 
       Dict_entity[i] = list(entity_finder(i,data_list))  


In [0]:
def Create_process_entity(core : int = multiprocessing.cpu_count() ) -> list : 
    manager = multiprocessing.Manager()
    global Dict_entity
    Dict_entity = manager.dict() 
    
    process_list=[]
    size = len(data_list) // core 
    step = core 
    for i in range(core): 
        if i != core :
            proc=multiprocessing.Process(target=parallel_entity, args=(int(size*i),int(size*(i+1))))  
        else : 
            proc=multiprocessing.Process(target=parallel_entity, args=(int(size*i),len(data_list)))    

        process_list.append(proc)  
    
    return process_list

In [0]:
Entity_process = Create_process_entity()
for proc in Entity_process: 
    proc.start()

In [0]:
def Checking_process():
    for proc in Entity_process:  
        alive = 'is dead'  
        if proc.is_alive() == True : 
            alive= 'is alive'

        print(f'{proc.name}  {alive}') 

    first = len(Dict_entity) 
    time.sleep(1) 
    vitesse=(len(Dict_entity)-first)
    print(f'{vitesse} corpus traité par seconde ' ) 
    print(f'avancement = {(len(Dict_entity) / len(data_list))*100 } %')

In [0]:
ENTITY= pd.DataFrame(Dict_entity.items(),columns=['num','tupl']).set_index('num')

In [0]:
vector_e=ENTITY.sort_index().values
entity_list=[]
for i in vector_e : 
    entity_list += i[0]

In [0]:
container = list(zip(*entity_list))

In [0]:
from collections import Counter

In [0]:
entity_count= Counter(container[0])

In [0]:
data_entité = list(zip(*entity_count.most_common()))

In [0]:
data_final_NER = pd.DataFrame() 
data_final_NER['entité']=data_entité[0]
data_final_NER['freq']=data_entité[1]

In [0]:
data_final_NER.to_csv('entité.csv')